# Elastic QnA

БД `Elasticsearch` может быть легко подключена к GigaChain. В данном примере рассмотрим, как можно использовать Elasticsearch для поиска ответов на вопросы по документам.

## Установка

Нужно, чтобы у вас был доступ к Elasticsearch. Для этого можно использовать [Elastic Cloud](https://www.elastic.co/cloud/) или запустить его локально.

```bash
    docker run -p 9200:9200 -e "discovery.type=single-node" -e "xpack.security.enabled=false" -e "xpack.security.http.ssl.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.9.0
```

In [17]:
# !pip install elasticsearch openai tiktoken gigachain

# Загрузка документов в БД

## Индексация и загрузка в БД

In [3]:
import getpass
import os

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import ElasticsearchStore

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
embeddings = OpenAIEmbeddings()

### Нарезаем документ на небольшие кусочки
Документ - статья из Википедд о крушении Титаника. Размер статьи около 100 кб. Мы будем разрезать её на кусочки по 500 символов.

In [43]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)

loader = TextLoader("../../modules/titanic_wiki_ru.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

print(f"Total docs: {len(docs)}")

Total docs: 331


### Загружаем все документы в Elastic, попутно вычисляя эмбединги

In [61]:
# docs = retriever.get_relevant_documents(query="Титаник", lang="ru")
db = ElasticsearchStore.from_documents(
    docs,
    embeddings,
    es_url="http://localhost:9200",
    index_name="test-basic",
)

db.client.indices.refresh(index="test-basic")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

### Question Answering on facts

In [51]:
from langchain_community.chat_models.gigachat import GigaChat

user = getpass.getpass("Giga user:")
password = getpass.getpass("Giga password:")

In [52]:
giga = GigaChat(profanity=False, credentials=...)

In [53]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_llm(llm=giga, retriever=db.as_retriever(earch_kwargs={"k": 5}))

In [58]:
questions = [
    "В каком году утонул титаник?",
    "Сколько человек погибло на титанике?",
    "Почему было так много жертв?",
    "Какой корабль первым пришел на помощь?",
    "Какова была мощность двигателей титаника?",
    "Чем отличались каюты разных классов?",
    "Что там было с биноклем?",
]
chat_history = []

for question in questions:
    result = qa(question)
    print(f"Q: {question}")
    print(f"A: {result['result']}\n")

Q: В каком году утонул титаник?
A: «Титаник» затонул в ночь с 14 на 15 апреля 1912 года.

Q: Сколько человек погибло на титанике?
A: По последним доступным данным, на борту «Титаника» погибло около 1500 человек.

Q: Почему было так много жертв?
A: Крушение "Титаника" стало одним из самых трагических событий в истории мореплавания. Причиной этого стали несколько факторов. Во-первых, корабль был построен без должной безопасности и не соответствовал современным стандартам. Во-вторых, он столкнулся с айсбергом во время сильного шторма, что привело к его разрушению и потере жизней всех пассажиров и экипажа. Кроме того, на борту находилось большое количество людей, которые не были должным образом подготовлены к такому большому количеству пассажиров и не имели достаточного количества спасательных средств. Все эти факторы вместе привели к гибели множества людей.

Q: Какой корабль первым пришел на помощь?
A: Первым на помощь пришел корабль "Карпатия".

Q: Какова была мощность двигателей титаник